In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [4]:
import json
with open('/content/drive/MyDrive/data_full.json') as json_file:
    CLINC150 = json.load(json_file)
CLINC150_train=CLINC150['train']
CLINC150_test=CLINC150['test']
CLINC150_val=CLINC150['val']


In [5]:
all_classes = {"card_declined"}
for c in CLINC150_train:
 all_classes.add(str(c[1]))

In [6]:
# classes =['transactions', 'thank_you', 'reminder_update', 'meeting_schedule', 'to-do list', 'order', 'make_call', 'credit_limit', 'exchange_rate', 'insurance', 'international_fees', 'last_maintenance', 'insurance_change', 'application_status', 'date', 'schedule_meeting', 'pin_change', 'expiration_date', 'are_you_a_bot', 'card_declined', 'credit_score', 'new_card', 'income', 'calendar', 'next_holiday', 'user name', 'pay_bill','freeze account', 'balance', 'replacement_card_duration', 'schedule_maintenance', 'improve_credit_score', 'taxes', 'bill_balance', 'credit_limit_change', 'account_blocked', 'transfer', 'order_checks', 'interest_rate', 'bill_due', 'rewards_balance', 'direct_deposit','change_language', 'change_user_name', 'spending_history'  ]

In [7]:
classes = ['accept_reservations' , 'greeting' ,  'card_declined', 
           'credit_limit', 'redeem_rewards' ,'freeze_account' ,
           'pay_bill' , 'cancel_reservation' ,  'yes' ,
           'change_language' ,'reminder' ,  'spending_history' , 'account_blocked' ,
           'meeting_schedule' ,  'application_status' , 'schedule_meeting' ,  'order_status' , 'credit_limit_change',
           'thank_you' , 'order' , 'confirm_reservation','reset_settings' ,'calendar' , 'change_user_name' ,  'calories' ,'report_fraud' ,
           'min_payment' , 'transactions' , 'rewards_balance' , 'what_can_i_ask_you', 'nutrition_info' ,'international_fees' ,
           'balance' ,'bill_balance', 'cancel', 'income', 'goodbye','bill_due']

In [8]:
train_data=[]
test_data=[]
val_data=[]

for c in CLINC150_train:
    if c[1] in classes:
        train_data.append(c)

for c in CLINC150_test:
    if c[1] in classes:
        test_data.append(c)

for c in CLINC150_val:
    if c[1] in classes:
        val_data.append(c)

In [9]:
import pandas as pd
df = pd.DataFrame(train_data)
df.to_csv('train_data.csv', index=False,header=('text','intent'))
train=pd.read_csv('train_data.csv')

df = pd.DataFrame(test_data)
df.to_csv('test_data.csv', index=False,header=('text','intent'))
test=pd.read_csv('test_data.csv')

df = pd.DataFrame(val_data)
df.to_csv('val_data.csv', index=False,header=('text','intent'))
valid=pd.read_csv('val_data.csv')

train = train.append(valid).reset_index(drop=True)

<ipython-input-9-2cea75b621bf>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(valid).reset_index(drop=True)


In [10]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
     


In [12]:
my_dict = {}
for i in range(len(classes)):
  my_dict[classes[i]] = i

def update_cat(x):
    return my_dict[x]

test['intent'] = test['intent'].apply(lambda x: update_cat(x))
train['intent'] = train['intent'].apply(lambda x: update_cat(x))



In [13]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = str(self.data.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.intent[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [14]:
print("TRAIN Dataset: {}".format(train.shape))
print("TEST Dataset: {}".format(test.shape))

training_set = Triage(train, tokenizer, MAX_LEN)
testing_set = Triage(test, tokenizer, MAX_LEN)

TRAIN Dataset: (4560, 2)
TEST Dataset: (1140, 2)


In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
     

In [16]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 38)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
     

In [17]:
model = DistillBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [18]:
print(len(classes))

38


In [19]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [20]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [21]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [22]:
for epoch in range(EPOCHS):
    train(epoch)
     

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 3.71234130859375
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 0: 35.5921052631579
Training Loss Epoch: 2.9348093065253473
Training Accuracy Epoch: 35.5921052631579
Training Loss per 5000 steps: 2.0369973182678223
Training Accuracy per 5000 steps: 75.0
The Total Accuracy for Epoch 1: 80.5701754385965
Training Loss Epoch: 1.1996878306070964
Training Accuracy Epoch: 80.5701754385965


In [23]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0 ; tr_loss=0 ;nb_tr_steps = 0;nb_tr_examples=0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [24]:

print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)
     

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 1.3031288385391235
Validation Accuracy per 100 steps: 50.0
Validation Loss Epoch: 0.7560659352363202
Validation Accuracy Epoch: 85.08771929824562
Accuracy on test data = 85.09%


In [25]:
def get_intent(event, max_length=512):
  input_text = "%s </s>" % event
  text = input_text
  text = " ".join(text.split())
  inputs = tokenizer.encode_plus(
      text,
      None,
      add_special_tokens=True,
      max_length=max_length,
      pad_to_max_length=True,
      return_token_type_ids=True,
      truncation=True
  )

  ids= torch.tensor(inputs['input_ids'], dtype=torch.long)
  mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)

  ids = ids.to(device, dtype = torch.long)
  mask = mask.to(device, dtype = torch.long)
 
  outputs = model(ids, mask).squeeze()
  return classes[torch.argmax(outputs).item()]


In [26]:
event = "how to pay electricity bill "
get_intent(event)

'pay_bill'

In [27]:
# import pickle
# with open("model1.pkl", "wb") as fp:
#     pickle.dump(model.state_dict(), fp)


In [28]:
# pickle.dump(model, open('model.pkl', 'wb'))

# event = pickle.load(open('model.pkl','rb'))
# pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))